In [1]:
import requests
import pandas as pd
import numpy as np
import io
import os
import geopandas as gpd
from datetime import timedelta, datetime, date
import time
from shapely.geometry import Point

In [6]:
to_day = time.localtime()
# time.strftime(format='%Y-%M-%D', t=to_day)
print(to_day)

time.struct_time(tm_year=2023, tm_mon=5, tm_mday=28, tm_hour=15, tm_min=45, tm_sec=39, tm_wday=6, tm_yday=148, tm_isdst=0)


In [12]:
to_day = datetime.now()
to_day.strftime('%Y-%m-%d')

'2023-05-28'

In [17]:
to_day = date.today()
ytd = to_day - timedelta(days=1)
print(to_day)
print(ytd)

2023-05-28
2023-05-27


In [20]:
to_day.strftime('%Y-%m-%d')

'2023-05-28'

In [4]:
gdf_thailand = gpd.read_file(r'E:\_CoopProject\data\thailand_gis-main\tambon\thailand_province_amphoe_tambon_simplify')
gdf_thailand

,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49453 13.75759, 100.49494 13.757..."
1,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.74800, 100.50412 13.747..."
2,Wat Ratchabophit,วัดราชบพิธ,TH100103,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.74800, 100.49633 13.748..."
3,Samran Rat,สำราญราษฎร์,TH100104,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50554 13.75378, 100.50412 13.747..."
4,San Chaopho Suea,ศาลเจ้าพ่อเสือ,TH100105,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49875 13.75560, 100.49821 13.751..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7420,Phadung Mat,ผดุงมาตร,TH961203,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.71286 6.17046, 101.71403 6.16538..."
7421,Chang Phueak,ช้างเผือก,TH961204,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.67997 5.99753, 101.67567 5.98895..."
7422,Chuap,จวบ,TH961301,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.86883 6.29824, 101.86908 6.29436..."
7423,Bu Kit,บูกิต,TH961302,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.88570 6.19545, 101.87473 6.19872..."


## Get Data from Nasa Firm

In [25]:
def getData():
    # get from url
    to_day = date.today()
    ytd = to_day - timedelta(days=1)
    to_day_str = to_day.strftime('%Y-%m-%d')
    ytd_str = ytd.strftime('%Y-%m-%d')
    url = f'https://firms2.modaps.eosdis.nasa.gov/api/country/csv/cea584916717f2288aa22711dadd89e7/MODIS_NRT/THA/2/{ytd_str}'
    urlData = requests.get(url).content
    fire_detect = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
    gdf = gpd.GeoDataFrame(fire_detect)

    # edit time column
    gdf['acq_time'] = gdf['acq_time'].astype('str')
    gdf['acq_time'] = gdf['acq_time'].apply(lambda x: x.rjust(4, '0'))

    gdf['acq_date_time'] = pd.to_datetime(gdf['acq_date'] + ' ' + gdf['acq_time'])
    gdf['th_date_time'] = gdf['acq_date_time'] + timedelta(hours=7)
    # gdf = gdf[(gdf['th_date_time']>=ytd_str) & (gdf['th_date_time']<=to_day_str)].reset_index().drop(columns=['index', 'acq_date_time'])
    gdf = gdf[(gdf['th_date_time']>=to_day_str)].reset_index().drop(columns=['index', 'acq_date_time'])
    gdf['th_date'] = gdf['th_date_time'].apply(lambda x: str(x).split(' ')[0])
    gdf['th_time'] = gdf['th_date_time'].apply(lambda x: str(x).split(' ')[1])
    gdf.drop(columns='th_date_time', inplace=True)

    # Create Point from lat, long
    geometry = [Point(xy) for xy in zip(gdf['longitude'], gdf['latitude'])]
    gdf['geometry'] = geometry
    gdf = gpd.GeoDataFrame(gdf, crs=4326)
    gdf['th_date'] = gdf['th_date'].astype('str')
    gdf['th_time'] = gdf['th_time'].astype('str')
    print(gdf.info())
    
    gdf.to_file(f'C:/coop_project_fire_spot/Fire-Spot-Thailand/data/{to_day_str}', encoding='utf-8', index=False)
    


getData()
# data = data.loc[:, ['latitude', 'longitude', 'confidence', 'th_date', 'th_time', 'geometry']]


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 0 entries
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   country_id  0 non-null      object  
 1   latitude    0 non-null      float64 
 2   longitude   0 non-null      float64 
 3   brightness  0 non-null      float64 
 4   scan        0 non-null      float64 
 5   track       0 non-null      float64 
 6   acq_date    0 non-null      object  
 7   acq_time    0 non-null      object  
 8   satellite   0 non-null      object  
 9   instrument  0 non-null      object  
 10  confidence  0 non-null      int64   
 11  version     0 non-null      object  
 12  bright_t31  0 non-null      float64 
 13  frp         0 non-null      float64 
 14  daynight    0 non-null      object  
 15  th_date     0 non-null      object  
 16  th_time     0 non-null      object  
 17  geometry    0 non-null      geometry
dtypes: float64(7), geometry(1), int64(1), object(9)
memory

c:\Users\chayo\anaconda3\envs\st_realtime\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


In [26]:
gpd.read_file(r'C:\coop_project_fire_spot\Fire-Spot-Thailand\data\2023-06-18')

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,th_date,th_time,geometry


## Mapping Tambon

In [27]:
def mapping_tambon():
    to_day = date.today()
    to_day_str = to_day.strftime('%Y-%m-%d')

    tambon_df = gpd.read_file(f'E:/_CoopProject/data/thailand_gis-main/tambon/thailand_province_amphoe_tambon_simplify')
    fire_spot_df = gpd.read_file(f'C:/coop_project_fire_spot/Fire-Spot-Thailand/data/{to_day_str}')
    fire_spot_df = gpd.overlay(fire_spot_df, tambon_df, how='intersection')

    fire_spot_df.to_file(f'C:/coop_project_fire_spot/Fire-Spot-Thailand/data/{to_day_str}', encoding='utf-8', index=False)

mapping_tambon()
mt = gpd.read_file(r'C:\coop_project_fire_spot\Fire-Spot-Thailand\data\2023-06-18')
mt


c:\Users\chayo\anaconda3\envs\st_realtime\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry


In [32]:
mt = gpd.read_file(r'C:\coop_project_fire_spot\Fire-Spot-Thailand\data\2023-06-18')
list_prov = list(mt['ADM1_TH'].unique())
list_prov



[]

In [33]:
mt.columns

Index(['country_id', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'th_date', 'th_time',
       'ADM3_EN', 'ADM3_TH', 'ADM3_PCODE', 'ADM2_EN', 'ADM2_TH', 'ADM2_PCODE',
       'ADM1_EN', 'ADM1_TH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_TH', 'ADM0_PCODE',
       'geometry'],
      dtype='object')

## Mapping Land Use

In [19]:
mt = gpd.read_file(f'C:/coop_project_fire_spot/Fire-Spot-Thailand/data/2023-06-17')

In [21]:
d = gpd.GeoDataFrame()
d.columns

Index([], dtype='object')

In [37]:
mt = gpd.read_file(f'C:/coop_project_fire_spot/Fire-Spot-Thailand/data/2023-06-18')
list(mt['ADM1_TH'].unique())

[]

In [40]:
def mapping_landuse():
    # to_day = date.today()
    # to_day_str = to_day.strftime('%Y-%m-%d')
    to_day_str = '2023-06-17'

    mt = gpd.read_file(f'C:/coop_project_fire_spot/Fire-Spot-Thailand/data/{to_day_str}')
    # mt['landuse'] = pd.Series()

    list_prov = list(mt['ADM1_TH'].unique())
    if len(list_prov) != 0:
        gdf = gpd.GeoDataFrame()
        for prov in list_prov:
            gdf_prov = mt[mt['ADM1_TH']==prov]
            landuse_gdf = gpd.read_file(f'E:/_CoopProject/landuse_extract/{prov}/การใช้ที่ดิน')
            landuse_gdf = landuse_gdf.to_crs('EPSG:4326')
        # landuse_gdf = landuse_gdf.loc[:, ['LUL1_CODE', 'LUL2_CODE', 'LU_CODE', 'LU_DES_TH', 'geometry']]
        # landuse_gdf.rename(columns = {'LU_DES_TH':'LU_DES'}, inplace = True)
            gdf_prov = gpd.overlay(gdf_prov, landuse_gdf, how='intersection')
            gdf = pd.concat([gdf, gdf_prov]).sort_values('th_time')
        print(gdf.info())
        gdf.to_file(f'C:/coop_project_fire_spot/Fire-Spot-Thailand/data/{to_day_str}', encoding='utf-8', index=False)
    else:
        print('No Province Keys')
    # return gdf
        # print(idx, row['ADM1_TH'])


mapping_landuse()



<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3 entries, 0 to 0
Data columns (total 44 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   country_id  3 non-null      object  
 1   latitude    3 non-null      float64 
 2   longitude   3 non-null      float64 
 3   brightness  3 non-null      float64 
 4   scan        3 non-null      float64 
 5   track       3 non-null      float64 
 6   acq_date    3 non-null      object  
 7   acq_time    3 non-null      object  
 8   satellite   3 non-null      object  
 9   instrument  3 non-null      object  
 10  confidence  3 non-null      int64   
 11  version     3 non-null      object  
 12  bright_t31  3 non-null      float64 
 13  frp         3 non-null      float64 
 14  daynight    3 non-null      object  
 15  th_date     3 non-null      object  
 16  th_time     3 non-null      object  
 17  ADM3_EN     3 non-null      object  
 18  ADM3_TH     3 non-null      object  
 19  ADM3

In [41]:
mt = gpd.read_file(r'C:\coop_project_fire_spot\Fire-Spot-Thailand\data\2023-06-17')
mt

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,LU_DES_EN,LUL1_CODE,LUL2_CODE,LU_DES,Shape_Area,RAI,OBJECTID,Rai_1,Shape_Leng,geometry
0,THA,16.17651,99.72781,317.11,1.08,1.04,2023-06-17,0330,Terra,MODIS,...,Factory,U,U5,บ.เบียร์ไทย 1991 จำกัด มหาชน,1.178027e+07,7363.0,NaN,NaN,NaN,POINT (99.72781 16.17651)
1,THA,16.70495,103.08415,319.01,1.06,1.03,2023-06-17,0330,Terra,MODIS,...,"City, Town, Commercial",U,U1,อำเภอกระนวน,3.626304e+06,NaN,83946.0,2266.440183,30685.353846,POINT (103.08415 16.70495)
2,THA,18.43043,99.59433,324.48,1.15,1.07,2023-06-17,0330,Terra,MODIS,...,Active paddy field,A,A1,NaN,9.156781e+05,572.0,NaN,NaN,NaN,POINT (99.59433 18.43043)


In [42]:
mt.columns

Index(['country_id', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'th_date', 'th_time',
       'ADM3_EN', 'ADM3_TH', 'ADM3_PCODE', 'ADM2_EN', 'ADM2_TH', 'ADM2_PCODE',
       'ADM1_EN', 'ADM1_TH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_TH', 'ADM0_PCODE',
       'LU_ID_L1', 'LU_ID_L2', 'LU_ID_L3', 'LU_CODE', 'LU_DES_TH', 'LU_DES_EN',
       'LUL1_CODE', 'LUL2_CODE', 'LU_DES', 'Shape_Area', 'RAI', 'OBJECTID',
       'Rai_1', 'Shape_Leng', 'geometry'],
      dtype='object')

In [44]:
mt[['th_date', 'th_time', 'ADM3_EN', 'ADM3_TH', 'ADM2_EN', 'ADM2_TH', 'ADM1_EN', 'ADM1_TH', 'LU_DES_TH', 'LU_DES_EN', 'latitude', 'longitude']]

,th_date,th_time,ADM3_EN,ADM3_TH,ADM2_EN,ADM2_TH,ADM1_EN,ADM1_TH,LU_DES_TH,LU_DES_EN,latitude,longitude
0,2023-06-17,10:30:00,Khlong Khlung,คลองขลุง,Khlong Khlung,คลองขลุง,Kamphaeng Phet,กำแพงเพชร,โรงงานอุตสาหกรรม,Factory,16.17651,99.72781
1,2023-06-17,10:30:00,Nong Ko,หนองโก,Kranuan,กระนวน,Khon Kaen,ขอนแก่น,ตัวเมืองและย่านการค้า,"City, Town, Commercial",16.70495,103.08415
2,2023-06-17,10:30:00,Bunnak Phatthana,บุญนาคพัฒนา,Mueang Lampang,เมืองลำปาง,Lampang,ลำปาง,นาข้าว,Active paddy field,18.43043,99.59433


In [14]:
d.columns

Index(['country_id', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'th_date', 'th_time',
       'ADM3_EN', 'ADM3_TH', 'ADM3_PCODE', 'ADM2_EN', 'ADM2_TH', 'ADM2_PCODE',
       'ADM1_EN', 'ADM1_TH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_TH', 'ADM0_PCODE',
       'LU_ID_L1', 'LU_ID_L2', 'LU_ID_L3', 'LU_CODE', 'LU_DES_TH', 'LU_DES_EN',
       'LUL1_CODE', 'LUL2_CODE', 'LU_DES', 'Shape_Area', 'RAI', 'geometry',
       'OBJECTID', 'Rai', 'Shape_Leng'],
      dtype='object')

In [18]:
d[['latitude', 'longitude', 'confidence', 'th_date', 'th_time', 'ADM1_TH', 'LU_DES_TH']].reset_index().drop(columns='index')

,latitude,longitude,confidence,th_date,th_time,ADM1_TH,LU_DES_TH
0,16.17651,99.72781,60,2023-06-17,10:30:00,กำแพงเพชร,โรงงานอุตสาหกรรม
1,16.70495,103.08415,70,2023-06-17,10:30:00,ขอนแก่น,ตัวเมืองและย่านการค้า
2,18.43043,99.59433,70,2023-06-17,10:30:00,ลำปาง,นาข้าว


In [74]:
from shapely.geometry import Polygon

gdf1 = gpd.GeoDataFrame({'Name': ['Polygon 1'], 'geometry': [Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])]})
gdf2 = gpd.GeoDataFrame({'Name': ['Polygon 2'], 'geometry': [Polygon([(2, 2), (3, 2), (3, 3), (2, 3)])]})

# Concatenate the GeoDataFrames vertically (along the rows)
concatenated_gdf = pd.concat([gdf1, gdf2], ignore_index=True)
concatenated_gdf

,Name,geometry
0,Polygon 1,"POLYGON ((0.00000 0.00000, 1.00000 0.00000, 1...."
1,Polygon 2,"POLYGON ((2.00000 2.00000, 3.00000 2.00000, 3...."


In [75]:
type(concatenated_gdf)

geopandas.geodataframe.GeoDataFrame

In [68]:
r = gpd.read_file(r'E:\_CoopProject\landuse_extract\เชียงใหม่\การใช้ที่ดิน')
r

,LU_ID_L1,LU_ID_L2,LU_ID_L3,LU_CODE,LU_DES_TH,LU_DES_EN,LUL1_CODE,LUL2_CODE,LU_DES,Shape_Area,RAI,geometry
0,2.0,20.0,2001.0,A001,เกษตรผสมผสาน/ไร่นาสวนผสม,Integrated farm/Diversified farm,A,A0,NaN,49726.375142,31,"POLYGON ((465120.855 2043504.459, 465125.272 2..."
1,2.0,20.0,2001.0,A001,เกษตรผสมผสาน/ไร่นาสวนผสม,Integrated farm/Diversified farm,A,A0,NaN,21192.775194,13,"POLYGON ((452972.765 2065486.906, 452921.709 2..."
2,2.0,20.0,2001.0,A001,เกษตรผสมผสาน/ไร่นาสวนผสม,Integrated farm/Diversified farm,A,A0,NaN,99907.408015,62,"POLYGON ((501907.554 2069990.411, 501842.279 2..."
3,2.0,20.0,2001.0,A001,เกษตรผสมผสาน/ไร่นาสวนผสม,Integrated farm/Diversified farm,A,A0,NaN,107258.928919,67,"POLYGON ((518706.016 2076354.621, 518757.609 2..."
4,2.0,20.0,2001.0,A001,เกษตรผสมผสาน/ไร่นาสวนผสม,Integrated farm/Diversified farm,A,A0,NaN,100141.689110,63,"POLYGON ((516977.364 2076893.437, 517020.120 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...
147042,4.0,42.0,4203.0,W203,คลองชลประทาน,Irrigation canal,W,W2,NaN,7979.603134,5,"POLYGON ((526457.220 2206478.461, 526452.727 2..."
147043,4.0,42.0,4203.0,W203,คลองชลประทาน,Irrigation canal,W,W2,NaN,11716.901084,7,"POLYGON ((525274.119 2210044.529, 525208.261 2..."
147044,4.0,42.0,4203.0,W203,คลองชลประทาน,Irrigation canal,W,W2,NaN,25894.968720,16,"POLYGON ((526506.576 2211711.164, 526486.142 2..."
147045,4.0,42.0,4203.0,W203,คลองชลประทาน,Irrigation canal,W,W2,NaN,53849.379768,34,"POLYGON ((527255.254 2211371.551, 527260.805 2..."


In [6]:
new_data = gpd.overlay(data, gdf_thailand, how='intersection')
new_data.head()

,latitude,longitude,confidence,th_date,th_time,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry
0,17.46168,101.89087,60,2023-05-22,10:45:00,Na Din Dam,นาดินดำ,TH420109,Mueang Loei,เมืองเลย,TH4201,Loei,เลย,TH42,Thailand,ประเทศไทย,TH,POINT (101.89087 17.46168)
1,17.65386,100.00925,68,2023-05-22,10:45:00,Fai Luang,ฝายหลวง,TH530804,Laplae,ลับแล,TH5308,Uttaradit,อุตรดิตถ์,TH53,Thailand,ประเทศไทย,TH,POINT (100.00925 17.65386)
2,18.24682,99.53375,88,2023-05-22,10:45:00,Phrabat,พระบาท,TH520105,Mueang Lampang,เมืองลำปาง,TH5201,Lampang,ลำปาง,TH52,Thailand,ประเทศไทย,TH,POINT (99.53375 18.24682)
3,18.58789,99.05137,31,2023-05-22,10:45:00,Ban Klang,บ้านกลาง,TH510112,Mueang Lamphun,เมืองลำพูน,TH5101,Lamphun,ลำพูน,TH51,Thailand,ประเทศไทย,TH,POINT (99.05137 18.58789)
4,18.69384,100.23157,62,2023-05-22,10:45:00,Sa-Iap,สะเอียบ,TH540606,Song,สอง,TH5406,Phrae,แพร่,TH54,Thailand,ประเทศไทย,TH,POINT (100.23157 18.69384)


In [8]:
out_fc_path = os.path.join('C:\coop_project_fire_spot\Fire-Spot-Thailand\data', date_str)
os.makedirs(out_fc_path, exist_ok=True)
new_data.to_file(out_fc_path, encoding='utf-8')

In [4]:
prov = pd.read_excel(r'C:\coop_project_fire_spot\Fire-Spot-Thailand\data\all_tambons.xlsx', sheet_name='Sheet1', header=0)
prov

,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,Shape_Area
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000128
1,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000061
2,Wat Ratchabophit,วัดราชบพิธ,TH100103,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000018
3,Samran Rat,สำราญราษฎร์,TH100104,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000019
4,San Chaopho Suea,ศาลเจ้าพ่อเสือ,TH100105,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000013
...,...,...,...,...,...,...,...,...,...,...
7420,Phadung Mat,ผดุงมาตร,TH961203,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,0.004115
7421,Chang Phueak,ช้างเผือก,TH961204,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,0.031049
7422,Chuap,จวบ,TH961301,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,0.004813
7423,Bu Kit,บูกิต,TH961302,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,0.005915


In [6]:
prov[prov['ADM1_TH']=='กรุงเทพมหานคร']['ADM2_TH'].unique()

array(['พระนคร', 'ดุสิต', 'หนองจอก', 'บางรัก', 'บางเขน', 'บางกะปิ',
       'ปทุมวัน', 'ป้อมปราบศัตรูพ่า', 'พระโขนง', 'มีนบุรี', 'ลาดกระบัง',
       'ยานนาวา', 'สัมพันธวงศ์', 'พญาไท', 'ธนบุรี', 'บางกอกใหญ่',
       'ห้วยขวาง', 'คลองสาน', 'ตลิ่งชัน', 'บางกอกน้อย', 'บางขุนเทียน',
       'ภาษีเจริญ', 'หนองแขม', 'ราษฎร์บูรณะ', 'บางพลัด', 'ดินแดง',
       'บึงกุ่ม', 'สาทร', 'บางซื่อ', 'จตุจักร', 'บางคอแหลม', 'ประเวศ',
       'คลองเตย', 'สวนหลวง', 'จอมทอง', 'ดอนเมือง', 'ราชเทวี', 'ลาดพร้าว',
       'วัฒนา', 'บางแค', 'หลักสี่', 'สายไหม', 'คันนายาว', 'สะพานสูง',
       'วังทองหลาง', 'คลองสามวา', 'บางนา', 'ทวีวัฒนา', 'ทุ่งครุ',
       'บางบอน'], dtype=object)

In [7]:
a = gpd.read_file(r'E:\_CoopProject\data\thailand_gis-main\tambon\shapefiles\Chiang Rai')
b = gpd.read_file(r'E:\_CoopProject\cri\เชียงราย2563\ขอบเขตการปกครอง\tambon_เชียงราย', encoding='cp874')

In [10]:
b.head()

,OBJECTID,TAMBON_IDN,TAM_CODE,TAM_NAM_T,AMPHOE_IDN,AMP_CODE,AMPHOE_T,AMPHOE_E,PROV_CODE,PROV_NAM_T,REMARK,ORIG_FID,Shape_Leng,Shape_Area,PROV_NAM_E,geometry
0,4884,570101,01,ต.เวียง,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4871,8798.119475,3.426093e+06,Chiang Rai,"POLYGON ((588018.592 2202314.379, 588101.405 2..."
1,4885,570102,02,ต.รอบเวียง,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4872,44338.503533,5.147291e+07,Chiang Rai,"POLYGON ((594052.103 2203315.133, 594073.166 2..."
2,4886,570103,03,ต.บ้านดู่,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4873,63870.714700,1.032365e+08,Chiang Rai,"POLYGON ((590400.403 2224232.669, 588337.900 2..."
3,4887,570104,04,ต.นางแล,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4874,56945.683101,1.035695e+08,Chiang Rai,"POLYGON ((597013.040 2224748.922, 596738.039 2..."
4,4888,570105,05,ต.แม่ข้าวต้ม,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4875,59506.284462,1.055717e+08,Chiang Rai,"POLYGON ((605664.430 2223980.173, 605491.305 2..."


In [9]:
a['ADM3_PCODE'] = a['ADM3_PCODE'].apply(lambda x: x[2:])
a.head()

,Shape_Leng,Shape_Area,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry
0,0.099252,0.000312,Wiang,เวียง,570101,Mueang Chiang Rai,เมืองเชียงราย,TH5701,Chiang Rai,เชียงราย,TH57,Thailand,ประเทศไทย,TH,"POLYGON ((99.84097 19.91549, 99.84176 19.91525..."
1,0.456162,0.004313,Rop Wiang,รอบเวียง,570102,Mueang Chiang Rai,เมืองเชียงราย,TH5701,Chiang Rai,เชียงราย,TH57,Thailand,ประเทศไทย,TH,"POLYGON ((99.89883 19.92759, 99.89903 19.92700..."
2,0.382767,0.005533,Ban Du,บ้านดู่,570103,Mueang Chiang Rai,เมืองเชียงราย,TH5701,Chiang Rai,เชียงราย,TH57,Thailand,ประเทศไทย,TH,"POLYGON ((99.84842 19.99287, 99.84898 19.99176..."
3,0.445377,0.005632,Nang Lae,นางแล,570104,Mueang Chiang Rai,เมืองเชียงราย,TH5701,Chiang Rai,เชียงราย,TH57,Thailand,ประเทศไทย,TH,"POLYGON ((99.84788 20.04191, 99.84887 20.04148..."
4,0.637215,0.008429,Mae Khao Tom,แม่ข้าวต้ม,570105,Mueang Chiang Rai,เมืองเชียงราย,TH5701,Chiang Rai,เชียงราย,TH57,Thailand,ประเทศไทย,TH,"POLYGON ((99.94840 20.13613, 99.94842 20.13613..."


In [17]:
for i in a['ADM3_PCODE'].values:
    if i not in list(b['TAMBON_IDN'].values):
        print(i)

570121
570404


In [23]:
b[b['AMPHOE_T']=='อ.เมืองเชียงราย']

,OBJECTID,TAMBON_IDN,TAM_CODE,TAM_NAM_T,AMPHOE_IDN,AMP_CODE,AMPHOE_T,AMPHOE_E,PROV_CODE,PROV_NAM_T,REMARK,ORIG_FID,Shape_Leng,Shape_Area,PROV_NAM_E,geometry
0,4884,570101,01,ต.เวียง,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4871,8798.119475,3.426093e+06,Chiang Rai,"POLYGON ((588018.592 2202314.379, 588101.405 2..."
1,4885,570102,02,ต.รอบเวียง,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4872,44338.503533,5.147291e+07,Chiang Rai,"POLYGON ((594052.103 2203315.133, 594073.166 2..."
2,4886,570103,03,ต.บ้านดู่,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4873,63870.714700,1.032365e+08,Chiang Rai,"POLYGON ((590400.403 2224232.669, 588337.900 2..."
3,4887,570104,04,ต.นางแล,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4874,56945.683101,1.035695e+08,Chiang Rai,"POLYGON ((597013.040 2224748.922, 596738.039 2..."
4,4888,570105,05,ต.แม่ข้าวต้ม,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4875,59506.284462,1.055717e+08,Chiang Rai,"POLYGON ((605664.430 2223980.173, 605491.305 2..."
5,4889,570106,06,ต.แม่ยาว,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4876,93442.464381,2.364220e+08,Chiang Rai,"POLYGON ((582569.453 2218111.156, 582705.029 2..."
6,4890,570107,07,ต.สันทราย,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4877,22424.316233,1.870485e+07,Chiang Rai,"POLYGON ((587434.343 2197039.869, 587311.968 2..."
7,4891,570111,11,ต.แม่กรณ์,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4878,42167.511007,7.553775e+07,Chiang Rai,"POLYGON ((576529.760 2198060.118, 576547.487 2..."
8,4892,570112,12,ต.ห้วยชมภู,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4879,109589.121608,2.925827e+08,Chiang Rai,"POLYGON ((560170.105 2213996.138, 560229.102 2..."
9,4893,570113,13,ต.ห้วยสัก,5701,01,อ.เมืองเชียงราย,Amphoe Muang Chiang Mai,57,จ.เชียงราย,NaN,4880,61293.875266,1.826366e+08,Chiang Rai,"POLYGON ((598000.926 2192759.364, 598107.573 2..."


In [20]:
a[a['ADM3_PCODE'].isin(['570121', '570404'])]

,Shape_Leng,Shape_Area,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry
15,0.392665,0.007153,Tha Sut,ท่าสุด,570121,Mueang Chiang Rai,เมืองเชียงราย,TH5701,Chiang Rai,เชียงราย,TH57,Thailand,ประเทศไทย,TH,"POLYGON ((99.92840 20.11704, 99.92770 20.11455..."
31,0.405083,0.006422,Mae Loi,แม่ลอย,570404,Thoeng,เทิง,TH5704,Chiang Rai,เชียงราย,TH57,Thailand,ประเทศไทย,TH,"POLYGON ((100.06778 19.62627, 100.06776 19.625..."


In [1]:
from unrar import rarfile
import os

In [2]:
in_path = r"E:\_CoopProject\landuse"
list_rar = os.listdir(in_path)
out_path = r"E:\_CoopProject\landuse_extract"

for file_name in list_rar:
    # create rar object from Rarfile and pass our rar file as parameter
    myrar_file = rarfile.RarFile(os.path.join(in_path, file_name))

    # extract rar file by using extractall method by give it rar file as parameter 
    # and the path you want to exract in this rar
    rarfile.RarFile.extractall(myrar_file, out_path)